In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

# Third-party imports
import astropy.cosmology
from astropy import units as u
import hmf
import halomod

# Project imports
import pyhalomodel as halo
import pyhalomodel.cosmology as cosmology
import pyhalomodel.camb_stuff as camb_stuff

In [ ]:
# Halo mass range [Msun/h]
Mmin, Mmax = 1e5, 1e17
nM = 1024

# Wavenumbers [h/Mpc]
kmin, kmax = 1e-4, 1e1
nk = 1024

# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
ns = 0.96
sigma_8  = 0.8

# Halo model
#mass_function = 'Sheth & Tormen (1999)'
#mass_function = 'Sheth, Mo & Tormen (2001)'
mass_function = 'Tinker et al. (2010)'
Dv = 200.   # Only for Tinker
dc = 1.686

# halomod
use_CAMB_for_halomod = False

# pyhalomod
use_halomod_quantities_in_pyhalomodel = True # Otherwise get quantities from CAMB as usual

# Redshift
z = 0.

In [ ]:
# Halomod

# Set cosmology
Omega_m = Omega_b+Omega_c
cosm = astropy.cosmology.Flatw0waCDM(
    H0 = 100.*h,
    Om0 = Omega_m,
    Ob0 = Omega_b,
    w0 = -1.,
    Tcmb0 = 2.725*u.K, 
    Neff = 3.046,
    )

# Set trasfer function
transfer_model = {
    'sigma_8': sigma_8, 
    'n': ns,
    'lnk_min': np.log(kmin), 
    'lnk_max': np.log(kmax), 
    'dlnk': np.log(kmax/kmin)/nk
    }

if mass_function == 'Sheth & Tormen (1999)':
    hmf_model = hmf.mass_function.fitting_functions.ST
    bias_model = halomod.bias.ST99
    mdef_model = 'SOVirial'
    mdef_params = None
    Dv = cosmology.Dv_BryanNorman(Omega_m)
elif mass_function == 'Sheth, Mo & Tormen (2001)':
    hmf_model = hmf.mass_function.fitting_functions.SMT
    bias_model = halomod.bias.SMT01
    mdef_model = 'SOVirial'
    mdef_params = None
    Dv = cosmology.Dv_BryanNorman(Omega_m)
elif mass_function == 'Tinker et al. (2010)':
    hmf_model = hmf.mass_function.fitting_functions.Tinker10
    bias_model = halomod.bias.Tinker10
    mdef_model = 'SOMean'
    mdef_params = {'overdensity': Dv}
else:
    print('Mass function:', mass_function)
    raise ValueError('Mass function not recognised')

# Setup halomod
hmodel = halomod.HaloModel(
    Mmin = np.log10(Mmin),
    Mmax = np.log10(Mmax),
    dlog10m = np.log10(Mmax/Mmin)/nM,
    bias_model = bias_model,
    hmf_model = hmf_model,
    mdef_model = mdef_model,
    mdef_params = mdef_params,
    disable_mass_conversion = True,
    delta_c = dc,
    cosmo_model = cosm,
    z = z,
    transfer_model = hmf.density_field.transfer_models.CAMB if use_CAMB_for_halomod else hmf.density_field.transfer_models.EH,
    halo_profile_model = 'NFW',
    halo_concentration_model = 'Duffy08',
    halo_concentration_params = {'sample': 'full', 'a': 10.14, 'b': -0.081, 'c': -1.01},
    force_1halo_turnover = False,
    hm_logk_min = np.log10(kmin),
    hm_logk_max = np.log10(kmax),
    hm_dlog10k = np.log10(kmax/kmin)/nk,
    **transfer_model,
    )

In [ ]:
# Halomod parameter values
pprint(hmodel.parameter_values)

In [ ]:
# pyhalomodel

# Linear power
k = hmodel.k_hm.copy() # NOTE: Important
if use_halomod_quantities_in_pyhalomodel: # Use CAMB for the linear spectrum and for sigmaR
    Omega_m = Omega_c+Omega_b
    Pk_lin = hmodel.linear_power_fnc(hmodel.k)
else: # Get Linear power and sigmaR directly from hmod
    Pk_lin, camb_results, Omega_m, _, _ = camb_stuff.run([z], Omega_c, Omega_b, Omega_k, h, ns, sigma_8)
    Pk_lin = Pk_lin(z, k)

# Halo model
pyhmod = halo.model(z, Omega_m, name=mass_function, Dv=Dv, dc=dc, verbose=True)
M = hmodel.m.copy() # NOTE: Important
if use_halomod_quantities_in_pyhalomodel:
    sigmaM = hmodel.sigma.copy()
else:
    R = pyhmod.Lagrangian_radius(M)
    sigmaM = camb_results.get_sigmaR(R, hubble_units=True, return_R_z=False)[[z].index(z)]

# Halo profiles
rv = pyhmod.virial_radius(M)
c = halo.pyhalomodel.concentration(M, z, method='Duffy et al. (2008)', halo_definition='M200')
matter_profile = halo.pyhalomodel.matter_profile(k, M, rv, c, pyhmod.Om_m)

# Power
Pk_2h, Pk_1h, Pk_hm = pyhmod.power_spectrum(k, Pk_lin, M, sigmaM, {'m': matter_profile})

In [ ]:
plt.subplots(2, 1)
rmin, rmax = 0.95, 1.05

plt.subplot(2, 1, 1)
plt.loglog(M, hmodel.sigma, label='halomod')
plt.loglog(M, sigmaM, label='pyhalomod')
plt.xticks([])
plt.ylabel('$\sigma(M)$')
plt.legend()

plt.subplot(2, 1, 2)
plt.semilogx(M, hmodel.sigma/hmodel.sigma)
plt.semilogx(M, sigmaM/hmodel.sigma)
plt.xlabel('$M$ [$h^{-1}M_\odot$]')
plt.ylabel('$\sigma(M)/\sigma_\mathrm{halomod}(M)$')
plt.ylim((rmin, rmax))

plt.show()

In [ ]:
plt.subplots(2, 1)
rmin, rmax = -1., 1.
multiplicity_hmodel = M*hmodel.dndlnm/pyhmod.rhom
multiplicity_pyhmod = pyhmod.multiplicity_function(M, sigmaM)


plt.subplot(2, 1, 1)
plt.semilogx(M, multiplicity_hmodel, label='halomod')
plt.semilogx(M, multiplicity_pyhmod, label='pyhalomod')
plt.xticks([])
plt.ylabel('$N(M)$')
plt.legend()

plt.subplot(2, 1, 2)
plt.semilogx(M, 100.*(-1.+multiplicity_hmodel/multiplicity_hmodel))
plt.semilogx(M, 100.*(-1.+multiplicity_pyhmod/multiplicity_hmodel))
plt.xlabel('$M$ [$h^{-1}M_\odot$]')
plt.ylabel('$N(M)/N_\mathrm{halomod}(M)-1$ [%]')
plt.ylim((rmin, rmax))

plt.show()

In [ ]:
# Initialise
plt.subplots(3, 1, sharex=True, figsize=(7, 7))
Pkmin, Pkmax = 1e1, 1e5
rmin, rmax = 0.7, 1.3
#smin, smax = 0.7, 1.3
smin, smax = 0.95, 1.05

# Power
plt.subplot(3, 1, 1)
plt.plot(np.nan, color='black', ls='-', label='Model')
plt.plot(np.nan, color='black', ls='--', label='Two-halo')
plt.plot(np.nan, color='black', ls=':', label='One-halo')
plt.loglog(k, Pk_lin, color='grey', label='Linear')
plt.loglog(k, hmodel.nonlinear_power, label='HALOFIT', color='C2')
plt.loglog(k, hmodel.power_auto_matter, label='halomod', color='C0')
plt.loglog(k, hmodel.power_2h_auto_matter, ls='--', color='C0')
plt.loglog(k, hmodel.power_1h_auto_matter, ls=':', color='C0')
plt.loglog(k, Pk_hm['m-m'], label='pyhalomodel', color='C1')
plt.loglog(k, Pk_2h['m-m'], ls='--', color='C1')
plt.loglog(k, Pk_1h['m-m'], ls=':', color='C1')
plt.xticks([])
plt.ylabel('$P(k)$')
plt.ylim((Pkmin, Pkmax))
plt.legend(loc='lower left', ncol=2)

# Ratio with HALOFIT
plt.subplot(3, 1, 2)
plt.axhline(1., color='black')
plt.axvline(hmodel.k_hm[0], color='black', ls='--')
plt.axvline(hmodel.k_hm[-1], color='black', ls='--')
plt.axvspan(kmin, hmodel.k_hm[0], color='black', alpha=0.1)
plt.semilogx(k, Pk_lin/hmodel.nonlinear_power, color='grey')
plt.semilogx(k, hmodel.power_auto_matter/hmodel.nonlinear_power, color='C0')
plt.semilogx(k, Pk_hm['m-m']/hmodel.nonlinear_power, color='C1')
plt.semilogx(k, hmodel.nonlinear_power/hmodel.nonlinear_power, color='C2')
plt.xticks([])
plt.xlim((kmin, kmax))
plt.ylabel('$P(k)/P_\mathrm{HALOFIT}(k)$')
plt.ylim((rmin, rmax))

# Ratio with halomod
plt.subplot(3, 1, 3)
plt.axhline(1., color='black')
plt.axvline(hmodel.k_hm[0], color='black', ls='--')
plt.axvline(hmodel.k_hm[-1], color='black', ls='--')
plt.axvspan(kmin, hmodel.k_hm[0], color='black', alpha=0.1)
plt.semilogx(k, Pk_lin/hmodel.power_auto_matter, ls='-', color='grey')
plt.semilogx(k, Pk_hm['m-m']/hmodel.power_auto_matter, ls='-', color='C1')
plt.semilogx(k, Pk_2h['m-m']/hmodel.power_2h_auto_matter, ls='--', color='C1')
plt.semilogx(k, Pk_1h['m-m']/hmodel.power_1h_auto_matter, ls=':', color='C1')
plt.xlabel('$k/h\mathrm{Mpc}^{-1}$')
plt.xlim((kmin, kmax))
plt.ylabel('$P(k)/P_\mathrm{halomod}(k)$')
plt.ylim((smin, smax))

# Finalise
plt.tight_layout()
plt.show()

In [ ]:
# Derivatives
from pyhalomodel.utility import derivative_from_samples
dpk_hmodel, dpk_pyhmod = np.zeros_like(k), np.zeros_like(k)
for ik, _k in enumerate(k):
    dpk_hmodel[ik] = derivative_from_samples(np.log(_k), np.log(k), np.log(hmodel.power_auto_matter))
    dpk_pyhmod[ik] = derivative_from_samples(np.log(_k), np.log(k), np.log(Pk_hm['m-m']))

# Plot
plt.subplots(1, 1)
plt.semilogx(k, dpk_hmodel, label='halomod')
plt.semilogx(k, dpk_pyhmod, label='pyhalomodel')
plt.legend()
plt.xlabel('$k$ [$h\mathrm{Mpc}^{-1}$]')
plt.ylabel(r'$\mathrm{d}\ln P(k)/\mathrm{d}\ln k$')
plt.show()